In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import string

from models import ContinuousIncrementalRSA
from utils import *
import time
from tqdm import tqdm

import scipy.stats

In [2]:
OBJ = {'color':'red', 'shape':'dress', 'string':'red dress'}
adjectives = ["red", "blue"]
nouns = ["pin", "dress"]#, "bottle", "pen", "glasses"]
objects = get_all_objects(adjectives, nouns)
utterances = get_all_utterances(adjectives, nouns)
v_adj = 0.9
v_noun = 0.95
alpha = 5
utt_types = ['red', 'dress', 'red dress', 'dress red']

In [3]:
rsa = ContinuousIncrementalRSA(adjectives=adjectives,
                                  nouns=nouns,
                                  objects=objects,
                                  utterances=utterances,
                                  v_adj=v_adj,
                                  v_noun=v_noun,
                                  alpha=alpha)

In [4]:
p_red = rsa.incremental_pragmatic_speaker(OBJ, 'red')
p_dress = rsa.incremental_pragmatic_speaker(OBJ, 'dress')
p_red_dress = rsa.incremental_pragmatic_speaker(OBJ, 'red dress')
p_dress_red = rsa.incremental_pragmatic_speaker(OBJ, 'dress red')

print('P(\'red\'| red dress) =', p_red)
print('P(\'dress\'| red dress) =', p_dress)
print('P(\'red dress\'| red dress) =', p_red_dress)
print('P(\'dress red\'| red dress) =', p_dress_red)

P('red'| red dress) = 0.016801417320292736
P('dress'| red dress) = 0.02850679179165062
P('red dress'| red dress) = 0.41601972625359535
P('dress red'| red dress) = 0.5386552155216565


In [5]:
initial_counts = [1, 1, 1, 1]
counts_df = pd.DataFrame(zip(utt_types, initial_counts), columns=['utt_type', 'count'])
counts_df['probs'] = counts_df['count'] / counts_df['count'].sum()
counts_df

,utt_type,count,probs
0,red,1,0.25
1,dress,1,0.25
2,red dress,1,0.25
3,dress red,1,0.25


In [6]:
counts_df.loc[counts_df['utt_type'] == 'red dress', 'count'] += 1
counts_df['probs'] = counts_df['count'] / counts_df['count'].sum()
counts_df

,utt_type,count,probs
0,red,1,0.2
1,dress,1,0.2
2,red dress,2,0.4
3,dress red,1,0.2


In [8]:
p_red_dress = rsa.incremental_pragmatic_speaker({'color':'red', 'shape':'dress', 'string':'red dress'}, 'red dress')
p_dress_red = rsa.incremental_pragmatic_speaker({'color':'red', 'shape':'dress', 'string':'red dress'}, 'dress red')

print('P(\'red dress\'| red dress) =', p_red_dress)
print('P(\'dress red\'| red dress) =', p_dress_red)

P('red dress'| red dress) = 0.41601972625359535
P('dress red'| red dress) = 0.5386552155216565


In [ ]:
NUM_UTT_TYPES = 4
NUM_SIMS = 100
NUM_GENS = 300

runs = np.zeros(shape=(NUM_UTT_TYPES, NUM_SIMS, NUM_GENS))

for sim_idx in tqdm(range(NUM_SIMS)):
    start = time.time()
    counts = [1, 1, 1, 1]
    for gen_idx in range(NUM_GENS):
        
        p_red = rsa.incremental_pragmatic_speaker(OBJ, 'red', counts_df)
        p_dress = rsa.incremental_pragmatic_speaker(OBJ, 'dress')
        p_red_dress = rsa.incremental_pragmatic_speaker(OBJ, 'red dress')
        p_dress_red = rsa.incremental_pragmatic_speaker(OBJ, 'dress red')

        utterance_type_prior = [count/sum(counts) for count in counts]
        probs = [p_red, p_dress, p_red_dress, p_dress_red]        

        #print('P(\'red\'| red dress) =', p_red)
        #print('P(\'dress\'| red dress) =', p_dress)
        #print('P(\'red dress\'| red dress) =', p_red_dress)
        #print('P(\'dress red\'| red dress) =', p_dress_red)

        denom = sum([utterance_type_prior[i]*probs[i] for i in range(4)])

        indices = list(range(4))
        normalized_probs = [(utterance_type_prior[i]*probs[i])/denom for i in range(4)]

        sample = np.random.choice(indices, p=normalized_probs)
        counts[sample] += 1

        for utt_idx in range(4):
            runs[utt_idx, sim_idx, gen_idx] = normalized_probs[utt_idx]